In [1]:
from floodligt_functions import clean_metadata
from floodligt_functions import read_position_data_jsonl

import numpy as np
from pathlib import Path
import json
import io
import polars as pl
import floodlight
#from floodlight.io.secondspectrum import read_position_data_jsonl
from floodlight.io.secondspectrum import read_teamsheets_from_meta_json
import json
import tempfile
from typing import Union, Tuple, Dict
from pathlib import Path
from floodlight.core.pitch import Pitch
from floodlight.io.utils import get_and_convert
from floodlight.core.teamsheet import Teamsheet
from floodlight.core.xy import XY
from floodlight.core.code import Code
from floodlight.models.kinematics import AccelerationModel
from floodlight.models.kinematics import DistanceModel
from floodlight.models.kinematics import VelocityModel

In [2]:
filepath_position = Path("SecondSpectrum/Second Spectrum/Anderlecht - Club Brugge/tracking-produced.jsonl")

In [3]:
filepath_metadata = Path("SecondSpectrum/Second Spectrum/Anderlecht - Club Brugge/rsc-bru_metadata.json")

In [4]:
filepath_cleaned_metadata = Path("cleaned_metadata.json")

In [ ]:
meta_cleaned_path = Path("cleaned_metadata.json")

In [6]:
clean_metadata(filepath_metadata)

In [7]:
position_data = read_position_data_jsonl(
    filepath_position,
    filepath_cleaned_metadata
)

# Getting the data

In [8]:
#position_data

In [9]:
halfs = ["HT2"]#["HT1", "HT2"]
teams = ["Home", "Away"]

In [10]:
# get list of home players
# this will be used to match player with correct position data
home_players = [player for player in position_data[3]['Home']["player"]]

# get list of away players
away_players = [player for player in position_data[3]['Away']["player"]]

In [11]:
possession_status = {}

for half in halfs:
    half_possession = position_data[1][half].code

    # Convert np.nan to None so Polars can handle them as nulls
    half_possession = [x if isinstance(x, str) else None for x in half_possession]

    # Create DataFrame
    possession_status[half] = pl.DataFrame({
        f"{half} possession": half_possession
    })

In [12]:
ball_status = {}

for half in halfs:
    half_ball_status = position_data[2][half].code
    half_ball_status = [x if isinstance(x, str) else None for x in half_ball_status]
    ball_status[half] = pl.DataFrame({
        f"{half} ball status": half_ball_status
    })


In [13]:
xy_objects = position_data[0]

In [14]:
#xy_objects['HT1']['Home']#.xy to access the xy values

In [15]:
xy_objects['HT1']['Home'].xy

array([[-46.38,  -0.06,  -0.57, ...,    nan,    nan,    nan],
       [-46.36,  -0.06,  -0.56, ...,    nan,    nan,    nan],
       [-46.35,  -0.08,  -0.55, ...,    nan,    nan,    nan],
       ...,
       [   nan,    nan,    nan, ...,    nan,    nan,    nan],
       [   nan,    nan,    nan, ...,    nan,    nan,    nan],
       [   nan,    nan,    nan, ...,    nan,    nan,    nan]],
      shape=(3054601, 42))

In [16]:
# position_data

Kinematics Function --> acceleration 

In [17]:
# Dictionary to store results
acceleration_dfs = {}

for half in halfs:
    acceleration_dfs[half] = {}
    
    for team in teams:
        am = AccelerationModel()
        am.fit(xy_objects[half][team])
        acceleration = am.acceleration()
        np_array = acceleration.property
        #np_array = np.where(np.isnan(np_array), 0.1, np_array)
        schema = home_players if team == "Home" else away_players
        acceleration_dfs[half][team] = pl.DataFrame(np_array, schema=schema)
        

In [33]:
acceleration

PlayerProperty(property=array([[-1.09071978e+00, -6.32827124e-13, -3.12500000e+00, ...,
                    nan,             nan,             nan],
       [-5.45359890e-01, -1.45705167e+00, -3.12500000e+00, ...,
                    nan,             nan,             nan],
       [-1.50129370e+00, -1.45705167e+00, -1.56250000e+00, ...,
                    nan,             nan,             nan],
       ...,
       [            nan,             nan,             nan, ...,
                    nan,             nan,             nan],
       [            nan,             nan,             nan, ...,
                    nan,             nan,             nan],
       [            nan,             nan,             nan, ...,
                    nan,             nan,             nan]],
      shape=(2901801, 21)), name='acceleration', framerate=25)

In [88]:
acceleration_df = pl.concat(
    [acceleration_dfs['HT2']['Home'], acceleration_dfs['HT2']['Away'], ball_status['HT2'], possession_status["HT2"]], how="horizontal"
).filter(
    (pl.col("HT2 ball status") == "A") & (pl.col("HT2 possession") == "A")
).drop(
    ["HT2 ball status", "HT2 possession"]
).select(
    pl.all().abs().is_not_nan().mean()
)

# other expressions
#pl.all().max()
#pl.all().min()



In [89]:
acceleration_df

C. Coosemans,Amuzu,M. Rits,T. Leoni,K. Dolberg,K. Sardella,A. Dreyer,L. Dendoncker,J. Simić,Amando Lapage,Ali Maamar,L. Vázquez,Y. Verschaeren,N. Engwanda,Mads Kikkenborg,S. Edozie,Thorgan Hazard,T. Degreef,Thomas Foket,Timon Vanhoutte,Anas Tajaouart,S. Mignolet,Ordoñez,Ferran Jutglà,K. Sabbe,C. Talbi,H. Vanaken,B. Mechele,C. Tzolis,Maxim De Cuyper,Ardon Jashari,Raphael Onyedika,Bjorn Meijer,Michal Skoras,J. Spileers,C. Nielsen,Romero,H. Vetlesen,Siquet,Jackers,Romeo Vermant,Gustaf Nilsson
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1.0,0.0,0.656885,0.656885,0.0,1.0,0.437817,1.0,1.0,1.0,1.0,1.0,0.343115,0.0,0.0,1.0,0.562183,0.343115,0.0,0.0,0.0,1.0,0.817623,0.437817,0.437817,0.817623,1.0,1.0,0.899589,1.0,1.0,1.0,0.562183,0.182377,0.182377,0.0,0.0,0.100411,0.0,0.0,0.0,0.562183


In [32]:
pl.from_numpy(np_array)

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-1.09072,-6.3283e-13,-3.125,0.0,3.125,4.419417,2.7756e-13,9.8532e-14,-2.7756e-13,-1.542511,-1.250838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-0.54536,-1.457052,-3.125,-1.026335,1.5625,4.419417,2.7756e-13,1.214256,-1.3878e-13,-2.069618,-1.378147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-1.501294,-1.457052,-1.5625,-1.026335,-1.5625,4.8919e-14,0.0,1.214256,0.0,-1.298362,-0.465301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-1.501294,1.2768e-13,0.0,1.026335,0.647209,-2.856917,0.129316,-2.139818,0.0,2.2204e-13,0.752728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.501294,-1.443669,0.0,0.0,3.493856,1.284148,0.507118,-2.139818,0.0,2.0123e-13,-0.287427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
acceleration_dfs['HT2']['Home']['M. Rits']

M. Rits
f64
-1.830583
-0.915291
-0.647209
-0.647209
-6.9389e-14
…
NaN
NaN
NaN


In [28]:
acceleration_dfs['HT2']['Home'].select(pl.col('M. Rits').is_not_nan().mean())

M. Rits
f64
0.017928


In [25]:
# concatenation attempt to see if dataframes are actujally concatenable
acc_ht1_away_df = pl.concat([acceleration_dfs['HT2']['Away'], ball_status['HT2'], possession_status["HT2"]], how="horizontal")

In [51]:
acc_ht1_home_df = pl.concat([acceleration_dfs['HT2']['Home'], ball_status['HT2'], possession_status["HT2"]], how="horizontal")

KeyError: 'Home'

In [26]:
acc_ht1_away_df

S. Mignolet,Ordoñez,Ferran Jutglà,K. Sabbe,C. Talbi,H. Vanaken,B. Mechele,C. Tzolis,Maxim De Cuyper,Ardon Jashari,Raphael Onyedika,Bjorn Meijer,Michal Skoras,J. Spileers,C. Nielsen,Romero,H. Vetlesen,Siquet,Jackers,Romeo Vermant,Gustaf Nilsson,HT2 ball status,HT2 possession
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str
-1.09072,-6.3283e-13,-3.125,0.0,3.125,4.419417,2.7756e-13,9.8532e-14,-2.7756e-13,-1.542511,-1.250838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""D""","""H"""
-0.54536,-1.457052,-3.125,-1.026335,1.5625,4.419417,2.7756e-13,1.214256,-1.3878e-13,-2.069618,-1.378147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""D""","""H"""
-1.501294,-1.457052,-1.5625,-1.026335,-1.5625,4.8919e-14,0.0,1.214256,0.0,-1.298362,-0.465301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""D""","""H"""
-1.501294,1.2768e-13,0.0,1.026335,0.647209,-2.856917,0.129316,-2.139818,0.0,2.2204e-13,0.752728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""D""","""H"""
1.501294,-1.443669,0.0,0.0,3.493856,1.284148,0.507118,-2.139818,0.0,2.0123e-13,-0.287427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""D""","""H"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,null,null
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,null,null
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,null,null


Kinematics Function --> distance model

In [ ]:
# Dictionary to store results
distance_dfs = {}

for half in halfs:
    distance_dfs[half] = {}
    
    for team in teams:
        dm = DistanceModel()
        dm.fit(xy_objects[half][team])
        distance = dm.distance_covered()
        np_array = distance.property
        schema = home_players if team == "Home" else away_players
        distance_dfs[half][team] = pl.DataFrame(np_array, schema=schema)
        

: 

In [ ]:
# Dictionary to store results
distance_dfs = {}

for half in halfs:
    distance_dfs[half] = {}
    
    for team in teams:
        dm = DistanceModel()
        dm.fit(xy_objects[half][team])
        distance = dm.cumulative_distance_covered()
        np_array = distance.property
        schema = home_players if team == "Home" else away_players
        distance_dfs[half][team] = pl.DataFrame(np_array, schema=schema)

In [22]:
distance_dfs

{'HT1': {'Away': shape: (3_054_601, 21)
  ┌─────────────┬──────────┬───────────────┬──────────┬───┬────────┬─────────┬─────────┬─────────┐
  │ S. Mignolet ┆ Ordoñez  ┆ Ferran Jutglà ┆ K. Sabbe ┆ … ┆ Siquet ┆ Jackers ┆ Romeo   ┆ Gustaf  │
  │ ---         ┆ ---      ┆ ---           ┆ ---      ┆   ┆ ---    ┆ ---     ┆ Vermant ┆ Nilsson │
  │ f64         ┆ f64      ┆ f64           ┆ f64      ┆   ┆ f64    ┆ f64     ┆ ---     ┆ ---     │
  │             ┆          ┆               ┆          ┆   ┆        ┆         ┆ f64     ┆ f64     │
  ╞═════════════╪══════════╪═══════════════╪══════════╪═══╪════════╪═════════╪═════════╪═════════╡
  │ 0.01        ┆ 0.01     ┆ 0.070711      ┆ 0.01     ┆ … ┆ NaN    ┆ NaN     ┆ NaN     ┆ NaN     │
  │ 0.007071    ┆ 0.007071 ┆ 0.075664      ┆ 0.01118  ┆ … ┆ NaN    ┆ NaN     ┆ NaN     ┆ NaN     │
  │ 0.01118     ┆ 0.005    ┆ 0.086313      ┆ 0.014142 ┆ … ┆ NaN    ┆ NaN     ┆ NaN     ┆ NaN     │
  │ 0.014142    ┆ 0.005    ┆ 0.097082      ┆ 0.014142 ┆ … ┆ NaN    ┆ 

Kinematics Function --> velocity model

In [ ]:
# Dictionary to store results
velocity_dfs = {}

for half in halfs:
    velocity_dfs[half] = {}
    
    for team in teams:
        vm = VelocityModel()
        vm.fit(xy_objects[half][team])
        velocity = vm.velocity()
        np_array = velocity.property
        schema = home_players if team == "Home" else away_players
        velocity_dfs[half][team] = pl.DataFrame(np_array, schema=schema)

# Analyzing the data properly

In [27]:
acc_ht1_away_df

S. Mignolet,Ordoñez,Ferran Jutglà,K. Sabbe,C. Talbi,H. Vanaken,B. Mechele,C. Tzolis,Maxim De Cuyper,Ardon Jashari,Raphael Onyedika,Bjorn Meijer,Michal Skoras,J. Spileers,C. Nielsen,Romero,H. Vetlesen,Siquet,Jackers,Romeo Vermant,Gustaf Nilsson,HT2 ball status,HT2 possession
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str
-1.09072,-6.3283e-13,-3.125,0.0,3.125,4.419417,2.7756e-13,9.8532e-14,-2.7756e-13,-1.542511,-1.250838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""D""","""H"""
-0.54536,-1.457052,-3.125,-1.026335,1.5625,4.419417,2.7756e-13,1.214256,-1.3878e-13,-2.069618,-1.378147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""D""","""H"""
-1.501294,-1.457052,-1.5625,-1.026335,-1.5625,4.8919e-14,0.0,1.214256,0.0,-1.298362,-0.465301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""D""","""H"""
-1.501294,1.2768e-13,0.0,1.026335,0.647209,-2.856917,0.129316,-2.139818,0.0,2.2204e-13,0.752728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""D""","""H"""
1.501294,-1.443669,0.0,0.0,3.493856,1.284148,0.507118,-2.139818,0.0,2.0123e-13,-0.287427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""D""","""H"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,null,null
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,null,null
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,null,null


In [ ]:
possession = ["H", "A"]
ball = ["D", "A"]


away_HT1 = {}

for possession_status in possession:
    away_HT1[possession_status] = {}
    
    for ball_status in ball:
        away_HT1[possession_status][ball_status] = acc_ht1_away_df.filter(
            (pl.col("HT2 possession")==possession_status) & (pl.col("HT2 ball status")==ball_status)
        )

In [29]:
away_HT1["H"]["D"].head()

S. Mignolet,Ordoñez,Ferran Jutglà,K. Sabbe,C. Talbi,H. Vanaken,B. Mechele,C. Tzolis,Maxim De Cuyper,Ardon Jashari,Raphael Onyedika,Bjorn Meijer,Michal Skoras,J. Spileers,C. Nielsen,Romero,H. Vetlesen,Siquet,Jackers,Romeo Vermant,Gustaf Nilsson,HT2 ball status,HT2 possession
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str
-1.09072,-6.3283e-13,-3.125,0.0,3.125,4.419417,2.7756e-13,9.8532e-14,-2.7756e-13,-1.542511,-1.250838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""D""","""H"""
-0.54536,-1.457052,-3.125,-1.026335,1.5625,4.419417,2.7756e-13,1.214256,-1.3878e-13,-2.069618,-1.378147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""D""","""H"""
-1.501294,-1.457052,-1.5625,-1.026335,-1.5625,4.8919e-14,0.0,1.214256,0.0,-1.298362,-0.465301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""D""","""H"""
-1.501294,1.2768e-13,0.0,1.026335,0.647209,-2.856917,0.129316,-2.139818,0.0,2.2204e-13,0.752728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""D""","""H"""
1.501294,-1.443669,0.0,0.0,3.493856,1.284148,0.507118,-2.139818,0.0,2.0123e-13,-0.287427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""D""","""H"""


In [19]:
away_HT1["H"]["D"]

NameError: name 'away_HT1' is not defined

In [30]:
away_HT1["H"]["D"].select([
    pl.col(col).abs().mean().alias(f"{col}_min") for col in away_players
])

S. Mignolet_min,Ordoñez_min,Ferran Jutglà_min,K. Sabbe_min,C. Talbi_min,H. Vanaken_min,B. Mechele_min,C. Tzolis_min,Maxim De Cuyper_min,Ardon Jashari_min,Raphael Onyedika_min,Bjorn Meijer_min,Michal Skoras_min,J. Spileers_min,C. Nielsen_min,Romero_min,H. Vetlesen_min,Siquet_min,Jackers_min,Romeo Vermant_min,Gustaf Nilsson_min
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
away_HT1["H"]["D"].select([
    pl.col(col).abs().max().alias(f"{col}_max") for col in away_players
])

S. Mignolet_max,Ordoñez_max,Ferran Jutglà_max,K. Sabbe_max,C. Talbi_max,H. Vanaken_max,B. Mechele_max,C. Tzolis_max,Maxim De Cuyper_max,Ardon Jashari_max,Raphael Onyedika_max,Bjorn Meijer_max,Michal Skoras_max,J. Spileers_max,C. Nielsen_max,Romero_max,H. Vetlesen_max,Siquet_max,Jackers_max,Romeo Vermant_max,Gustaf Nilsson_max
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
7.807764,6.981456,7.961471,6.437364,6.593038,8.575156,6.415378,6.798234,5.871191,9.07594,7.8808,9.504194,9.191229,7.809523,NaN,NaN,9.10613,NaN,NaN,NaN,9.308768


In [35]:
test = acc_ht1_away_df.select(pl.max(away_players))

In [36]:
test.shape

(1, 21)

In [48]:
test_df = test.transpose(include_header=True)
test_df

column,column_0
str,f64
"""S. Mignolet""",9.35004
"""Ordoñez""",9.569555
"""Ferran Jutglà""",8.19405
"""K. Sabbe""",9.856817
"""C. Talbi""",10.004882
…,…
"""H. Vetlesen""",7.578342
"""Siquet""",NaN
"""Jackers""",NaN


In [49]:

test_df.rename({"column": "player_name", "column_0": "metric"})

player_name,metric
str,f64
"""S. Mignolet""",9.35004
"""Ordoñez""",9.569555
"""Ferran Jutglà""",8.19405
"""K. Sabbe""",9.856817
"""C. Talbi""",10.004882
…,…
"""H. Vetlesen""",7.578342
"""Siquet""",NaN
"""Jackers""",NaN
